<a href="https://colab.research.google.com/github/alexhmyang/AIface/blob/master/ollama_colab_runner.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ollama Colab Runner**
# <img src='https://ollama.com/public/ollama.png' alt="Ollama"/>
When running this, ideally, select an instance with GPU:<br>
T4 for free ones, A100/L4 for paid subscribers<br><br>
Run each of the 3 cells, before running your prompt.<br>
If you interrupt execution, start the server again

In [1]:
# @title Install components
!curl https://ollama.ai/install.sh | sh
!pip install ollama

!echo 'debconf debconf/frontend select Noninteractive' | sudo debconf-set-selections
!sudo apt-get update && sudo apt-get install -y cuda-drivers

import os
# Set LD_LIBRARY_PATH so the system NVIDIA library
os.environ.update({'LD_LIBRARY_PATH': '/usr/lib64-nvidia'})

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  27541      0 --:--:-- --:--:-- --:--:-- 27553
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.
Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [347 B]
Get:5 https://develope

In [2]:
# @title Start server
import subprocess
proccess = subprocess.Popen(['ollama', 'serve'])

In [8]:
# @title Select your model
model = "qwen3:4b-q8_0" # @param ["qwen3:4b-q8_0","qwen3:14b","deepseek-r1:1.5b","deepseek-r1:7b","deepseek-r1:14b","deepseek-r1:32b","deepseek-r1:70b","deepseek-coder:1.3b","deepseek-coder:6.7b","deepseek-coder:33b","gemma3:12b","gemma3:27b","llama3.3:70b","mistral:7b","phi4:14b","qwen2.5:7b","qwen2.5:14b","qwen2.5:32b","qwen2.5-coder:7b","qwen2.5-coder:14b","qwen2.5-coder:32b"]
!ollama pull {model}

In [11]:
# @title Interacting with the model
question = "写一篇200字的报告" # @param {"type":"string"}
from IPython.display import display, Markdown, HTML
import ollama
import time
import re

def count_chinese_characters(text):
    """Counts the number of Chinese characters in a string."""
    # This is a simple regex for common Chinese character ranges.
    # It might not be exhaustive but covers most cases.
    chinese_chars = re.findall(r'[\u4e00-\u9fff]', text)
    return len(chinese_chars)

start_time = time.time()
# Use streaming for real-time output
stream = ollama.chat(model=model, messages=[
  {
    'role': 'user',
    'content': question,
  },
], stream=True)

generated_text = ""
output_div = display(HTML("<div id='output'></div>"), display_id=True)

for chunk in stream:
    if chunk['message']['content'] is not None:
        generated_text += chunk['message']['content']
        # Update the output div with the current generated text
        output_div.update(HTML(f"<div id='output'>{generated_text}</div>"))

end_time = time.time()
generation_time = end_time - start_time

# Assuming each Chinese character is roughly equivalent to one token for a simplified calculation
# For more accurate token counting, a proper tokenizer for the model would be needed.
token_count = count_chinese_characters(generated_text)
tokens_per_second = token_count / generation_time if generation_time > 0 else 0

print(f"\nGeneration time: {generation_time:.2f} seconds")
print(f"Tokens generated: {token_count}")
print(f"Speed: {tokens_per_second:.2f} tokens/second")

print(model)


Generation time: 11.80 seconds
Tokens generated: 587
Speed: 49.75 tokens/second
qwen3:4b-q8_0
